![logo_ironhack_blue 7](https://user-images.githubusercontent.com/23629340/40541063-a07a0a8a-601a-11e8-91b5-2f13e4e6b441.png)

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np

# Loading CSV File for CLeaning

In [ ]:
recipes = pd.read_csv('../data/raw/recipes_clean.csv')

In [ ]:
recipes.head(5)

# Start Cleaning Columns

In [ ]:
recipes.columns = recipes.columns.str.lower()

In [ ]:
recipes.columns

In [ ]:
recipes.columns = recipes.columns.str.replace(' ', '_')

In [ ]:
recipes.columns

In [ ]:
recipes['meal_class'].unique()

In [ ]:
mapping_rules = {'Prato-principal': ' ', 'Peixes, Prato-principal':'Peixes',
       'Saladas': 'Saladas', 'Doces e Sobremesas': 'Doces e Sobremesas', 'Refeição Principal': ' ', 'Sopas':'Sopas',
       'Bebidas': 'Bebidas', 'Entradas e Petiscos':'Entradas e Petiscos', 'Carnes, Peixes': ' ',
       'Peixes, Saladas':'Peixes', 'alimentação saudável, receitas fáceis': ' ',
       'receitas fáceis': ' ', 'receitas de peixe': 'Peixes', 'receita de frango': 'Carnes',
       'Carnes, Prato-principal': 'Carnes', 'Carnes, Entradas e Petiscos': ' ',
       'Entradas e Petiscos, Sopas': ' ', 'Entradas e Petiscos, Peixes': ' ',
       'Acompanhamentos': 'Acompanhamentos', 'Doces e Sobremesas, Entradas e Petiscos': ' ',
       'Entradas e Petiscos, Sopas, Vegetariano': ' ', 'alimentação saudável': ' ',
       'Entradas e Petiscos, Vegetariano':'Entradas e Petiscos',
       'Acompanhamentos, Entradas e Petiscos': 'Acompanhamentos',
       'alimentação saudável, receita com abobora': ' ',
       'Entradas e Petiscos, Prato-principal': ' ',
       'Prato-principal, Vegetariano': 'Vegetariano', 'Saladas, Vegetariano': 'Vegetariano',
       'Vegetariano':'Vegetariano', 'Entradas e Petiscos, Saladas': 'Entradas e Petiscos',
       'Acompanhamentos, Doces e Sobremesas': ' ',
       'Acompanhamentos, Vegetariano': 'Acompanhamentos', 'Bebidas, Doces e Sobremesas': 'Bebidas, Doces e Sobremesas',
       'Prato-principal, Saladas': 'Saladas', 'Acompanhamentos, Carnes': ' ',
       'Sopas, Vegetariano': 'Sopas', 'Doces e Sobremesas, Peixes, Sopas': ' ',
       'Carnes, Doces e Sobremesas': ' ', 'Doces e Sobremesas, Peixes': ' ',
       'Carnes, Saladas': 'Carnes', 'Acompanhamentos, Entradas e Petiscos, Saladas': ' ',
       'Acompanhamentos, Saladas': 'Acompanhamentos',
       'Carnes, Doces e Sobremesas, Entradas e Petiscos': ' ',
       'Acompanhamentos, Doces e Sobremesas, Entradas e Petiscos': ' ',
       'Acompanhamentos, Peixes': ' ',
       'Doces e Sobremesas, Entradas e Petiscos, Peixes': ' ',
       'Doces e Sobremesas, Prato-principal': ' '}

In [ ]:
recipes['meal_class'] = recipes['meal_class'].map(mapping_rules)

In [ ]:
recipes['meal_class'].unique()

In [ ]:
recipes['meal_class'] = recipes['meal_class'].astype(str)

recipes = recipes[~recipes['meal_class'].str.contains('Bebidas')]

recipes.reset_index(drop=True, inplace=True)

In [ ]:
recipes['meal_class'].unique()

In [ ]:
unique_meal_counts = recipes['meal_class'].value_counts()
unique_meal_counts

In [ ]:
recipes.info()

In [ ]:
recipes['difficulty'].unique()

In [ ]:
unique_difficulty_counts = recipes['difficulty'].value_counts()
unique_difficulty_counts

In [ ]:
recipes['cost'].unique()

In [ ]:
mapping = {'Económico': 'Económico', 'Médio': 'Médio', 'Dispendioso': 'Dispendioso', 'Económico e Médio': 'Económico'}

In [ ]:
recipes['cost'] = recipes['cost'].map(mapping)

In [ ]:
recipes['cost'].unique()

In [ ]:
unique_costs_counts = recipes['cost'].value_counts()
unique_costs_counts

In [ ]:
recipes['rating'].unique()

In [ ]:
def clean_column(df, column_name, string_to_remove):
    """
    Clean a specific column in a DataFrame by removing a specified string and rounding to integers.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the column to be cleaned.
    string_to_remove (str): The string to be removed from the column values.

    Returns:
    pd.DataFrame: A new DataFrame with the specified column cleaned, rounded to integers, and non-finite values filled with 0.
    """
    # Check if the column contains non-string values and convert them to strings
    df[column_name] = df[column_name].astype(str)
    
    # Remove the specified string from the specified column
    df[column_name] = df[column_name].str.replace(string_to_remove, '')
    
    # Convert to float and handle empty strings by replacing them with NaN
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')
    
    # Round the values to the nearest integer and fill non-finite values (NaN or inf) with 0
    df[column_name] = df[column_name].round().fillna(0).astype(int)
    
    return df

In [ ]:
column_name = 'rating'
string_to_remove = ' out of 5'
recipes = clean_column(recipes, column_name, string_to_remove)

In [ ]:
recipes['rating'].unique()

In [ ]:
unique_rating_counts = recipes['rating'].value_counts()
unique_rating_counts

In [ ]:
def convert_time_columns_to_minutes(df, time_columns):
    """
    Convert time columns in a DataFrame to minutes.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    time_columns (list): A list of column names containing time values in 'Xh Ymin' format.

    Returns:
    pd.DataFrame: A modified DataFrame with the time columns converted to minutes.
    """
    
    # Copy the input DataFrame to avoid modifying the original
    modified_df = df.copy()
    
    for col in time_columns:
        # Split the time string into hours and minutes
        time_parts = modified_df[col].str.split('h ')
        
        # Convert hours and minutes to integers
        hours = time_parts.str[0].astype(int)
        minutes = time_parts.str[1].str.replace('min', '').astype(int)
        
        # Calculate the total time in minutes and replace the column
        total_minutes = hours * 60 + minutes
        modified_df[col] = total_minutes
    
    return modified_df

In [ ]:
time_columns = ['prep_time', 'total_time']
recipes = convert_time_columns_to_minutes(recipes, time_columns)

In [ ]:
recipes['prep_time'].unique()

In [ ]:
recipes['total_time'].unique()

In [ ]:
unique_prep_counts = recipes['prep_time'].value_counts()
unique_prep_counts

In [ ]:
unique_total_time_counts = recipes['total_time'].value_counts()
unique_total_time_counts

In [ ]:
rows_with_smaller_time = recipes[recipes['total_time'] < recipes['prep_time']]

if not rows_with_smaller_time.empty:
    print("Rows with 'total_time' smaller than 'prep_time':")
    for index, row in rows_with_smaller_time.iterrows():
        print(f"Title: {row['title']}, Total Time: {row['total_time']}, Prep Time: {row['prep_time']}")
else:
    print("All 'total_time' values are greater than or equal to 'prep_time'.")

In [ ]:
def keep_larger_time_column(df):
    """
    Keep only one column between 'prep_time' and 'total_time' based on the larger value.

    Parameters:
    df (pd.DataFrame): The input DataFrame with 'prep_time' and 'total_time' columns.

    Returns:
    pd.DataFrame: A new DataFrame with one column 'time' containing the larger time values.
    """
    # Create a new 'time' column containing the larger values between 'prep_time' and 'total_time'
    df['time'] = df[['prep_time', 'total_time']].max(axis=1)

    # Drop the original 'prep_time' and 'total_time' columns
    df.drop(['prep_time', 'total_time'], axis=1, inplace=True)

    return df

In [ ]:
recipes = keep_larger_time_column(recipes)

In [ ]:
recipes

In [ ]:
position = 6

columns = list(recipes.columns)

columns.remove('time')

columns.insert(position, 'time')

recipes = recipes[columns]

recipes

In [ ]:
recipes.rename(columns={'time': 'time(min)'}, inplace=True)

In [ ]:
recipes['time(min)'].unique()

In [ ]:
unique_time_counts = recipes['time(min)'].value_counts()
unique_time_counts

In [ ]:
recipes['ingredient_name'].unique()

In [ ]:
recipes['ingredient_unit'].unique()

# Saving CSV File with Clean Dataframe

In [ ]:
recipes

In [ ]:
recipes.to_csv('../data/raw/recipes2.csv', index = False)

# Creating new Dataframe with only one line per Recipe

In [ ]:
recipes_copy = recipes.copy()

In [ ]:
recipes_copy['ingredients_combined'] = (
    recipes_copy['ingredient_amount'].astype(str) +
    ' ' +
    recipes_copy['ingredient_unit'] +
    ' ' +
    recipes_copy['ingredient_name']
)
recipes_copy['ingredients_combined'] = recipes_copy['ingredients_combined'].str.replace('-', '')
# Group by 'title',
recipes_copy = recipes_copy.groupby(['title', 'servings', 'preparations', 'meal_class', 'difficulty', 'cost', 'rating', 
                                    'time(min)', 'recipe_link', 'image_url'])['ingredients_combined'].apply(list).reset_index()

In [ ]:
recipes_copy

In [ ]:
recipes_copy = recipes_copy[['meal_class', 'title', 'rating', 'cost', 'difficulty', 'servings','ingredients_combined', 'preparations', 'time(min)', 'recipe_link', 'image_url']]

In [ ]:
recipes_copy

In [ ]:
recipes_copy.to_csv('../data/raw/recipes_one_line.csv', index = False)